<a href="https://colab.research.google.com/github/nivedita1989/autocorr/blob/master/Petition_Coreference_entity_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''#!python -m spacy download en_core_web_lg
# NLTK
#!pip install pycorenlp
import re
import nltk
from nltk.corpus import wordnet, sentiwordnet
# Spacy
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load('en_core_web_lg') # For word and sentence level tokenization and POS tagging
from pycorenlp import StanfordCoreNLP
'''
import nltk
nltk.download('wordnet')
nltk.download('sentiwordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


True

In [ ]:
f=open('TestSentAna.txt','r')
text=f.read()
#text

In [ ]:
"""Making NEGATORS word list"""
NEGATORS = ["ain't",'cannot',"can't","didn't","doesn't","don't","hadn't",'hardly',
            "hasn't","haven't","havn't","isn't",'lack','lacking','lacks','neither',
            'never','no','nobody','none','nor','not','nothing','nowhere',"mightn't",
            "mustn't","needn't","oughtn't","shouldn't","wasn't",'without',"wouldn't",
            'are not','can not', 'did not','does not','do not','had not', 'has not',
            'have not','is not','might not','must not','need not','ought not','should not',
            'was not','would not', "n't"]
NEGATORS.sort(key = lambda x: 1/len(x))


In [ ]:

f = open("TestSentAna.txt", "r")
text=f.read()
def text_clean(text):
  text = ' '+text+' '
  for negator in NEGATORS:
      text = text.replace(negator, '"not"')
  text = re.sub(r'\s+', ' ', text) # Remove repeated spaces/newlines
  text = re.sub(r'(<\^).*', '', text) # Remove anything after <^

  text = re.sub(r'\((.*?)\)',' ', text) # Remove ('text')
  text = re.sub(r'\[(.*?)\]',' ', text) # Remove ['text']
  text = re.sub(r'<(.*?)>',' ', text) # Remove <'text'>
  text = re.sub(r'\S*@\S*\s?', ' ', text) # Remove emails
  text = re.sub(r'http\S+', ' ', text) # Remove URLs

  text = re.sub(r'\s+', ' ', text) # Remove repeated spaces/newlines again
  text = text.replace(' *','.')
  text = text.replace('..','.')
  text = re.sub(r'\s+', ' ', text) # Remove repeated spaces/newlines again
  text = ' ' + text + ' ' # Add spaces surrounding the text, this makes finding whether or not a phrase occurs simpler
  text = text.lower()

  for c in ''':'.,";-''':
      text = text.replace(c, ' '+c+' ')
  text = re.sub(r'''[^a-zA-Z0-9,.;:/\-'"]+''', ' ', text) # Get rid of special characters, except some punctuation

  text = text.replace(" won ' t ", " won't ")
  text = text.replace('n t', 'not')

  return text
  ###########
newtext=text_clean(text)
text=newtext
print(text)


 petition made change with , rindani started petitionoto rajeev chandrasekhar and othersit seems like the popular ' beti padhao , beti bachao ' sloganothat we all love to chant does not cover all the young girls that need 13 . there is serious shortage of secondary and senior secondary schools for girls in 24 . data shows that in - , for every primary schools there are only secondary and senior secondary schools . shortage of schools ruins the future of lakhs of in 24 . 63 are forced to dropout and become child brides or teenage mothers , with a host of health problems . as a self - confident , educated woman who got to go to college , 44 want to ensure that every girl in 24 has a school to go to . sign 44 petition asking congress and the bjp to make this a priority in 50 election manifestos . moreschoolsforgirls when 65 re in a positionoto help someone less privileged than 65 , the first instinct for many of 65 is to ask two questions : if the person has children , and if children go 

In [ ]:
sentences = [str(s).replace('.','') for s in nlp(text).sents]
sentences
num_chain_dic={'65': ['us', 'we', 'we', 'us'], '73': ['The Right to Education ( RTE ) Act', 'it'], '105': ['their', 'the political parties'], '137': ['the country', 'our country'], '44': ['I', 'my'], '13': ['it', 'supporters!Aditi Rindani'], '141': ['This petition', 'my petition', 'my petition', 'this petition'], '143': ['we', 'our', 'we', 'our', 'we'], '50': ['their', 'the Congress and the BJP'], '24': ['India', 'India', 'India'], '90': ['them', 'boys and girls'], '27': ['This shortage of schools', 'a serious shortage of secondary ( Class and ) and senior secondary schools ( Class'], '123': ['it', 'the dropout rate of girls'], '63': ['They', 'those children', 'girls in India']}
entity_lis=num_chain_dic.keys()
entity_lis

dict_keys(['65', '73', '105', '137', '44', '13', '141', '143', '50', '24', '90', '27', '123', '63'])

In [ ]:
class Sentence(object):
    """
    """
    def __init__(self, text):
        self.raw = text
        self.text = text
        self.tokens = [token for token in nlp(self.text)] # word_tokenize(self.text)
        self.pos_tags = list(zip(map(str, self.tokens), map(lambda token: token.pos_, self.tokens))) # nltk.pos_tag(self.tokens)
        self.sent_tags = [(tup[0], word_to_sentiment(word=tup[0], pos=tup[1])) for tup in self.pos_tags]
        self.pos_and_sent_tags = [(self.pos_tags[i][0], self.pos_tags[i][1], self.sent_tags[i][1]) for i in range(len(self.pos_tags))]
        self.total_sent = list_to_score(self.pos_tags)
        self.posneg_count = posneg_count(self.pos_and_sent_tags)

    def parse(self):
        """
        Splits a sentence into a list of chunks comprising the sentence (uses the pos_and_sent_tags attribute)
        """
        # Find number of currencies in the sentence
        if unique_entity_count(self.pos_and_sent_tags) == 1: # If this number is 1, return the entire sentence
            return [self.pos_and_sent_tags]
        elif unique_entity_count(self.pos_and_sent_tags) == 0: # If this number is 0, return an empty list
            return []
        if unique_entity_count(self.pos_and_sent_tags) > 1: # If this number is > 1, parse on commas
            phrase_list = []
            tags_so_far = []
            for tup in self.pos_and_sent_tags:
                token = tup[0]
                if (token != ',') and (token != ';'):
                    tags_so_far.append(tup)
                elif (token == ',') or (token == ';'):
                    phrase_list.append(tags_so_far)
                    tags_so_far = []
            phrase_list.append(tags_so_far)
        if len(phrase_list) == 0:
            return phrase_list

        # Discard phrases with a unique_curr_count of 0
        phrase_list = list(filter(lambda x: unique_entity_count(x) > 0, phrase_list))

        def phrase_splitter(phrase):
            if unique_entity_count(phrase) > 1 and posneg_count(phrase) > 1:
                part1 = phrase[:1]
                i = 0
                while unique_entity_count(part1) == 0:
                    part1 = phrase[:i]
                    i += 1
                while posneg_count(part1) == 0:
                    part1 = phrase[:i]
                    i += 1

                part2 = phrase[i-1:]
                if unique_entity_count(part2) == 0:
                    return phrase
                else:
                    return [part1, part2]
            else:
                return [phrase]

        split_phrase_list = []
        for phrase in phrase_list:
            split_phrase_list.extend(phrase_splitter(phrase))

        return split_phrase_list

    def sent_dic_wordnet(self):

        chunks = self.parse()
        sent_dict_list = []

        for chunk in chunks:
            sent_dict = dict()
            sent = list_to_score(chunk)

            # Check if a negator is in the chunk. if so, reverse score
            text = ' '.join([tag[0] for tag in chunk])

            if ' not ' in ' '+text+' ':
                sent *= -0.9

            # for word in NEGATORS:
            #     if ' '+word+' ' in ' '+text+' ':
            #         sent *= -0.9 # Literal is to diminish how much negation flips sentiment
            #         break

            for entity in unique_entities(chunk):
                sent_dict[entity] = sent
            sent_dict_list.append(sent_dict)

        # Combine each dict in sent_dict_list, summing common entries
        combined_dict = dict()
        for entity in unique_entities(self.pos_and_sent_tags):
            combined_dict[entity] = sum([dic.get(entity, 0) for dic in sent_dict_list])

        return combined_dict


In [ ]:


def find_sent_dic(text):
    if type(text)!=str:
        return dict()
    S = Sentence(text)
    return S.sent_dic_wordnet()
    '''try:
        return S.sent_dic_wordnet()
    except:
        return dict()'''

In [ ]:
S=Sentence(sentences[7])
print(sentences[7])
#unique_entity_count(S.pos_and_sent_tags)
S.sent_dic_wordnet()


as a self - confident , educated woman who got to go to college , 44 want to ensure that every girl in 24 has a school to go to 


{'24': 0.025, '44': -0.075}

In [ ]:
def posneg_count(sent_tags):
    """
    Number of either positive or negative words in a list of pos_and_sent_tags
    """
    nonzero_sentiments = list(filter(lambda tup: tup[2] != 0, sent_tags))
    return len(nonzero_sentiments)

def unique_entities(pos_and_sent_tags):
    text = ' '.join([tup[0] for tup in pos_and_sent_tags])
    return set(filter(lambda c: c in text, entity_lis))

def unique_entity_count(pos_and_sent_tags):
    return len(unique_entities(pos_and_sent_tags))

In [ ]:
def word_to_sentiment(word, pos=None):
    """
    Given a word and an optional part of speech tag,
    returns the average sentiment score for that word using senti_synsets.
    """
    if word in STOP_WORDS or word in NEGATORS:
        return 0

    if pos == None:
        synsets = list(sentiwordnet.senti_synsets(word))

    elif pos != None:
        def get_wordnet_pos(tag):
            """
            Translating Spacy's POS tags to WordNet's POS tags
            """
            if tag.startswith('ADJ'): return wordnet.ADJ
            elif tag.startswith('VERB'): return wordnet.VERB
            elif tag.startswith('NOUN'): return wordnet.NOUN
            elif tag.startswith('ADV'): return wordnet.ADV
            else: return wordnet.NOUN

        wordnet_pos = get_wordnet_pos(pos)
        synsets = list(sentiwordnet.senti_synsets(word, wordnet_pos))

    score = 0
    for el in synsets:
        score += el.pos_score()
        score -= el.neg_score()

    if score == 0 and pos != None:
        return word_to_sentiment(word, pos=None)

    if len(synsets) == 0: return 0

    score /= len(synsets)
    return score

In [ ]:
def list_to_score(pos_tags):
    """
    Finds the average sentiment of a list of tuples of (word, pos) using word_to_sentiment()
    """
    score = 0
    entity_count = 0
    words_with_sentiment = 0
    for tup in pos_tags:
        word, pos = tup[0], tup[1]
        if word in entity_lis:
            entity_count += 1
        else:
            if word_to_sentiment(word, pos) != 0:
                words_with_sentiment += 1
                score += word_to_sentiment(word, pos)
    if score == 0:
        return 0
    else:
        return score/words_with_sentiment

[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/sentiwordnet.zip.


True

In [ ]:
sentences = [str(s).replace('.','') for s in nlp(text).sents]

sent_dict_list = [find_sent_dic(s) for s in sentences]


In [ ]:
# Combine dictionaries, summing common entries:
combined_dict = dict()
for entity in entity_lis:
    combined_dict[entity] = sum([dic.get(entity, 0) for dic in sent_dict_list])
combined_dict = {k:v for k,v in combined_dict.items() if v != 0}

sign = lambda x: 'Negative' if x < -.01 else 'Positive' if x > .01 else 'Neutral'
combined_dict = {k:sign(v) for k,v in combined_dict.items()}

In [ ]:
combined_dict


{'105': 'Negative',
 '13': 'Negative',
 '143': 'Positive',
 '24': 'Negative',
 '44': 'Negative',
 '50': 'Positive',
 '63': 'Positive',
 '65': 'Negative',
 '73': 'Positive',
 '90': 'Positive'}

In [ ]:
num_chain_dic

{'105': ['their', 'the political parties'],
 '123': ['it', 'the dropout rate of girls'],
 '13': ['it', 'supporters!Aditi Rindani'],
 '137': ['the country', 'our country'],
 '141': ['This petition', 'my petition', 'my petition', 'this petition'],
 '143': ['we', 'our', 'we', 'our', 'we'],
 '24': ['India', 'India', 'India'],
 '27': ['This shortage of schools',
  'a serious shortage of secondary ( Class and ) and senior secondary schools ( Class'],
 '44': ['I', 'my'],
 '50': ['their', 'the Congress and the BJP'],
 '63': ['They', 'those children', 'girls in India'],
 '65': ['us', 'we', 'we', 'us'],
 '73': ['The Right to Education ( RTE ) Act', 'it'],
 '90': ['them', 'boys and girls']}